In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## 데이터들의 의미
(https://www.kaggle.com/c/nyc-taxi-trip-duration/data)

id - a unique identifier for each trip

vendor_id - a code indicating the provider associated with the trip record

pickup_datetime - date and time when the meter was engaged

dropoff_datetime - date and time when the meter was disengaged

passenger_count - the number of passengers in the vehicle (driver entered value)

pickup_longitude - the longitude where the meter was engaged

pickup_latitude - the latitude where the meter was engaged

dropoff_longitude - the longitude where the meter was disengaged

dropoff_latitude - the latitude where the meter was disengaged

store_and_fwd_flag - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip


**target value : trip_duration - duration of the trip in seconds**

# Q1. Outlier가 데이터 셋에 존재할 경우 선형회귀에 대한 성능을 비교해보세요

In [2]:
data_df = pd.read_csv('./train.csv')

In [3]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null object
dropoff_datetime      1458644 non-null object
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


**모든 feature가 수치형 데이터임이 아님을 알 수 있다.    
id, vendor_id, store_and_fwd_flag는 고유값이므로 이에 대한 처리가 필요!**

In [4]:
np.round(data_df.describe(),2)

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00
mean,1.53,1.66,-73.97,40.75,-73.97,40.75,959.49
std,0.50,1.31,0.07,0.03,0.07,0.04,5237.43
min,1.00,0.00,-121.93,34.36,-121.93,32.18,1.00
25%,1.00,1.00,-73.99,40.74,-73.99,40.74,397.00
50%,2.00,1.00,-73.98,40.75,-73.98,40.75,662.00
75%,2.00,2.00,-73.97,40.77,-73.96,40.77,1075.00
max,2.00,9.00,-61.34,51.88,-61.34,43.92,3526282.00


**min, quantile, max를 살펴보면 어떤 outlier가 trip_duration에 있음을 알 수 있습니다.**

**vendor id, id는 각각 identify를 위한 데이터이므로 drop을 사용하여 처리합니다. pick up datetime, drop off datetime은 각각 시계열 데이터인데, 이 책에서 시계열 처리에 대해 다루지 않았으므로 drop을 해줍니다.**

In [5]:
#pandas의 drop 메서트를 활용하여 위의 feature를 drop해보세요
data_df.drop(['vendor_id', 'id', 'pickup_datetime', 'dropoff_datetime'], axis=1, inplace=True)

In [6]:
data_df.head(3)

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,1,-73.979027,40.763939,-74.005333,40.710087,N,2124


**store_and_fwd_flag와 trip_duration의 상관관계를 알아본다. 만약 상관관계가 적다면 drop하고 그렇지 않다면 핸들링을 따로 한다.**

In [7]:
sff_cat = {'N':0, 'Y':1}
cat_fun = lambda x:sff_cat[x]
sff = data_df['store_and_fwd_flag'].map(cat_fun)
data_df.groupby('store_and_fwd_flag')['trip_duration'].mean()

store_and_fwd_flag
N     958.819706
Y    1080.763331
Name: trip_duration, dtype: float64

**위의 결과를 보고 store_and_fwd_flag을 drop할지 하지 않을지 생각해보세요.**     
drop한다. 왜냐하면 `store_and_fwd_flag`가 `N`이든 `Y`이든 `trip_duration`의 평균값은 비슷하기 때문이다.

In [8]:
data_df.drop('store_and_fwd_flag', axis=1, inplace=True)

**trip_duration에 outlier 수를 파악해보고, 이 outlier들을 drop했을 때와 안했을 때의 성능 수준을 선형 회귀 모델들로 비교해 보겠습니다.**

In [9]:
np.sum(data_df.trip_duration > 10000)

2123

# outlier가 있는 데이터셋을 사용하여 선형 회귀 모델 평가.

In [10]:
#outlier를 제거하지 않은 데이터 셋을 훈련세트와 데이터 세트로 나눈다.
X = data_df.drop('trip_duration', axis = 1)
y = data_df.trip_duration

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

LinearRegression을 사용하여 train set을 학습시킨후 test set으로 평가해보세요

In [11]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_preds = lr.predict(X_test)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)

print('MSE : {0:.3f}, RMSE : {1:.3f}'.format(mse, rmse))

MSE : 10613951.781, RMSE : 3257.906


Pipeline과 PolynomialFeatures를 가지고 다항회귀를 사용해서 평가해보세요

In [12]:
degree = 4

polynomial_features = PolynomialFeatures(degree=degree, include_bias=False)
linear_regression = LinearRegression()
pipeline = Pipeline([('polynomial_features', polynomial_features), ('linear_regression', linear_regression)])
pipeline.fit(X, y)

scores = cross_val_score(pipeline, X, y, scoring='neg_mean_squared_error', cv=10)
mse = -1*np.mean(scores)
rmse = np.sqrt(mse)

print('MSE : {0:.3f}, RMSE : {1:.3f}'.format(mse, rmse))

MSE : 266975178838253.562, RMSE : 16339375.106


Ridge 모델 학습

다양한 learning rate 가지고 ridge 모델을 평가해보세요.

이 때 각각의 coefficient를 확인해보세요

In [13]:
alphas = [0, 0.1, 1, 10, 100]
coeff_df = pd.DataFrame()

for alpha in alphas :
    ridge = Ridge(alpha = alpha)
    ridge.fit(X, y)

    coeff = pd.Series(data=ridge.coef_ , index=X.columns)
    colname='alpha:'+str(alpha)
    coeff_df[colname] = coeff
    
    neg_mse_scores = cross_val_score(ridge, X, y, scoring="neg_mean_squared_error", cv = 5)
    avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
    
    print('alpah {0}일 때 평균 RMSE : {1:.3f}'.format(alpha, avg_rmse))
    
sort_column = 'alpha:'+str(alphas[0])
coeff_df.sort_values(by=sort_column, ascending=False)

alpah 0일 때 평균 RMSE : 4880.747
alpah 0.1일 때 평균 RMSE : 4880.747
alpah 1일 때 평균 RMSE : 4880.746
alpah 10일 때 평균 RMSE : 4880.741
alpah 100일 때 평균 RMSE : 4880.706


,alpha:0,alpha:0.1,alpha:1,alpha:10,alpha:100
pickup_longitude,2579.793034,2579.702827,2578.891289,2570.807235,2493.038537
passenger_count,32.842209,32.842247,32.842598,32.846079,32.879078
dropoff_longitude,-635.115468,-635.047255,-634.433634,-628.326654,-570.105529
dropoff_latitude,-1693.514743,-1693.527107,-1693.637549,-1694.660226,-1697.784999
pickup_latitude,-3700.613807,-3700.384937,-3698.326812,-3677.912100,-3488.897423


Lasso 모델 학습

다양한 learning rate 가지고 Lasso 모델을 평가해보세요.

이 때 각각의 coefficient를 확인해보세요

In [14]:
alphas = [0.07, 0.1, 0.5, 1, 3]
coeff_df = pd.DataFrame()

for alpha in alphas :
    lasso = Lasso(alpha = alpha)
    lasso.fit(X, y)

    coeff = pd.Series(data=lasso.coef_ , index=X.columns)
    colname='alpha:'+str(alpha)
    coeff_df[colname] = coeff
    
    neg_mse_scores = cross_val_score(lasso, X, y, scoring="neg_mean_squared_error", cv = 5)
    avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
    
    print('alpah {0}일 때 평균 RMSE : {1:.3f}'.format(alpha, avg_rmse))
    
sort_column = 'alpha:'+str(alphas[0])
coeff_df.sort_values(by=sort_column, ascending=False)

alpah 0.07일 때 평균 RMSE : 4880.745
alpah 0.1일 때 평균 RMSE : 4880.745
alpah 0.5일 때 평균 RMSE : 4880.788
alpah 1일 때 평균 RMSE : 4880.828
alpah 3일 때 평균 RMSE : 4881.814


,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3
pickup_longitude,2517.539415,2491.044507,2137.520033,1852.749009,1387.812874
passenger_count,32.817124,32.806346,32.662682,32.466365,31.609559
dropoff_longitude,-576.263618,-551.187581,-216.635969,-0.000000,0.000000
dropoff_latitude,-1656.301064,-1640.390050,-1428.190467,-1163.792958,-109.273686
pickup_latitude,-3667.511413,-3653.242980,-3463.111471,-3182.830707,-1879.171371


ElasticNet 모델 학습

다양한 learning rate 가지고 ElasticNet 모델을 평가해보세요.

이 때 각각의 coefficient를 확인해보세요

In [15]:
alphas = [0.07, 0.1, 0.5, 1, 3]
coeff_df = pd.DataFrame()

for alpha in alphas :
    elastic_net = ElasticNet(alpha = alpha)
    elastic_net.fit(X, y)

    coeff = pd.Series(data=elastic_net.coef_ , index=X.columns)
    colname='alpha:'+str(alpha)
    coeff_df[colname] = coeff
    
    neg_mse_scores = cross_val_score(elastic_net, X, y, scoring="neg_mean_squared_error", cv = 5)
    avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
    
    print('alpah {0}일 때 평균 RMSE : {1:.3f}'.format(alpha, avg_rmse))
    
sort_column = 'alpha:'+str(alphas[0])
coeff_df.sort_values(by=sort_column, ascending=False)

alpah 0.07일 때 평균 RMSE : 4884.411
alpah 0.1일 때 평균 RMSE : 4884.630
alpah 0.5일 때 평균 RMSE : 4885.131
alpah 1일 때 평균 RMSE : 4885.213
alpah 3일 때 평균 RMSE : 4885.304


,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3
pickup_longitude,234.914021,172.361288,37.358399,18.445435,5.543465
dropoff_longitude,113.659384,86.469639,19.784044,9.631381,2.598494
passenger_count,33.017631,32.744505,29.356658,25.952230,17.602133
dropoff_latitude,-106.452385,-75.009375,-14.475565,-6.754124,-1.588787
pickup_latitude,-137.681187,-97.111011,-19.004566,-9.026961,-2.348671


# outlier를 없엔 데이터 셋을 사용하여 선형 회귀모델 평가.

In [16]:
outlier_idx = y[y > 10000].index
data_dropped_outlier = data_df.drop(outlier_idx, axis = 0)

In [17]:
#데이터가 2000개 가량 줄어든 것을 확인할 수 있다.
data_dropped_outlier.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1456521 entries, 0 to 1458643
Data columns (total 6 columns):
passenger_count      1456521 non-null int64
pickup_longitude     1456521 non-null float64
pickup_latitude      1456521 non-null float64
dropoff_longitude    1456521 non-null float64
dropoff_latitude     1456521 non-null float64
trip_duration        1456521 non-null int64
dtypes: float64(4), int64(2)
memory usage: 77.8 MB


In [18]:
X_dropped = data_dropped_outlier.drop('trip_duration', axis = 1)
y_dropped = data_dropped_outlier.trip_duration

X_train, X_test, y_train, y_test = train_test_split(X_dropped, y_dropped, random_state = 42)

앞서서 했던 outlier를 제거하기 전 시행했던 각각의 모델들을 위의 데이터셋을 통해 평가해보세요.

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_preds = lr.predict(X_test)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)

print('MSE : {0:.3f}, RMSE : {1:.3f}'.format(mse, rmse))

MSE : 405081.596, RMSE : 636.460


In [27]:
degree = 4

polynomial_features = PolynomialFeatures(degree=degree, include_bias=False)
linear_regression = LinearRegression()
pipeline = Pipeline([('polynomial_features', polynomial_features), ('linear_regression', linear_regression)])
pipeline.fit(X_dropped, y_dropped)

scores = cross_val_score(pipeline, X_dropped, y_dropped, scoring='neg_mean_squared_error', cv=10)
mse = -1*np.mean(scores)
rmse = np.sqrt(mse)

print('MSE : {0:.3f}, RMSE : {1:.3f}'.format(mse, rmse))

MSE : 9106513900996.393, RMSE : 3017700.101


In [21]:
alphas = [0, 0.1, 1, 10, 100]
coeff_df = pd.DataFrame()

for alpha in alphas :
    ridge = Ridge(alpha = alpha)
    ridge.fit(X_dropped, y_dropped)

    coeff = pd.Series(data=ridge.coef_ , index=X_dropped.columns)
    colname='alpha:'+str(alpha)
    coeff_df[colname] = coeff
    
    neg_mse_scores = cross_val_score(ridge, X_dropped, y_dropped, scoring="neg_mean_squared_error", cv = 5)
    avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
    
    print('alpah {0}일 때 평균 RMSE : {1:.3f}'.format(alpha, avg_rmse))
    
sort_column = 'alpha:'+str(alphas[0])
coeff_df.sort_values(by=sort_column, ascending=False)

alpah 0일 때 평균 RMSE : 632.565
alpah 0.1일 때 평균 RMSE : 632.564
alpah 1일 때 평균 RMSE : 632.562
alpah 10일 때 평균 RMSE : 632.544
alpah 100일 때 평균 RMSE : 632.418


,alpha:0,alpha:0.1,alpha:1,alpha:10,alpha:100
pickup_longitude,2299.012403,2298.930449,2298.193188,2290.853136,2220.586730
passenger_count,6.655625,6.655658,6.655960,6.658967,6.687621
dropoff_longitude,-473.790165,-473.729799,-473.186807,-467.786412,-416.612563
dropoff_latitude,-1961.291765,-1961.257186,-1960.945680,-1957.801867,-1923.991790
pickup_latitude,-2945.983910,-2945.826321,-2944.409026,-2930.334427,-2798.603546


In [22]:
alphas = [0.07, 0.1, 0.5, 1, 3]
coeff_df = pd.DataFrame()

for alpha in alphas :
    lasso = Lasso(alpha = alpha)
    lasso.fit(X_dropped, y_dropped)

    coeff = pd.Series(data=lasso.coef_ , index=X_dropped.columns)
    colname='alpha:'+str(alpha)
    coeff_df[colname] = coeff
    
    neg_mse_scores = cross_val_score(lasso, X_dropped, y_dropped, scoring="neg_mean_squared_error", cv = 5)
    avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
    
    print('alpah {0}일 때 평균 RMSE : {1:.3f}'.format(alpha, avg_rmse))
    
sort_column = 'alpha:'+str(alphas[0])
coeff_df.sort_values(by=sort_column, ascending=False)

alpah 0.07일 때 평균 RMSE : 632.575
alpah 0.1일 때 평균 RMSE : 632.585
alpah 0.5일 때 평균 RMSE : 632.654
alpah 1일 때 평균 RMSE : 632.979
alpah 3일 때 평균 RMSE : 640.229


,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3
pickup_longitude,2236.544093,2209.999763,1855.876471,1697.286065,1232.593156
passenger_count,6.630734,6.620032,6.477378,6.268148,5.412899
dropoff_longitude,-414.756723,-389.636587,-54.544388,-0.000000,0.000000
dropoff_latitude,-1924.039405,-1908.120599,-1695.829138,-1432.081055,-377.934077
pickup_latitude,-2912.991426,-2898.750965,-2708.966437,-2394.336781,-1089.542352


In [23]:
alphas = [0.07, 0.1, 0.5, 1, 3]
coeff_df = pd.DataFrame()

for alpha in alphas :
    elastic_net = ElasticNet(alpha = alpha)
    elastic_net.fit(X_dropped, y_dropped)

    coeff = pd.Series(data=elastic_net.coef_ , index=X_dropped.columns)
    colname='alpha:'+str(alpha)
    coeff_df[colname] = coeff
    
    neg_mse_scores = cross_val_score(elastic_net, X_dropped, y_dropped, scoring="neg_mean_squared_error", cv = 5)
    avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
    
    print('alpah {0}일 때 평균 RMSE : {1:.3f}'.format(alpha, avg_rmse))
    
sort_column = 'alpha:'+str(alphas[0])
coeff_df.sort_values(by=sort_column, ascending=False)

alpah 0.07일 때 평균 RMSE : 653.037
alpah 0.1일 때 평균 RMSE : 654.271
alpah 0.5일 때 평균 RMSE : 657.085
alpah 1일 때 평균 RMSE : 657.499
alpah 3일 때 평균 RMSE : 657.797


,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3
pickup_longitude,214.988234,157.853691,34.207322,16.851206,5.007178
dropoff_longitude,111.112711,84.268143,19.150745,9.300722,2.485156
passenger_count,7.252894,7.195917,6.389421,5.562113,3.528989
dropoff_latitude,-104.531282,-73.614125,-14.177681,-6.603498,-1.537810
pickup_latitude,-118.560294,-83.605194,-16.256020,-7.648734,-1.887705


# Q2. 이 데이터 셋에서는 규제가 그리 효과적이지 못한 것 처럼 보입니다. 그 이유는 무엇일까요?
애초에 위에 있는 feature들이 `time_duration`과 큰 연관이 없어서 규제를 하더라도 효과적이지 못하다.

# Q3. 이 데이터셋은 우리가 원하는 trip duration을 구하기에는 연관이 적은 것 같습니다. 데이터를 가공한다면, 어떤식으로 하면 trip duration과 관련이 있을까요?
위에서 drop한 feature 중에 `pickup_datetime`과 `dropout_datetime`과의 연관성을 살펴보고 연관성이 높을 시 이를 feature로 두고 `trip_duration`의 예측값을 구해본다.

# 부록:파이썬의 시계열 데이터 

파이썬 라이브러리에는 날짜와 시간을 위한 자료형과 달력과 관련된 기능을 제공하는 자료형이 있습니다..

파이썬 기본 자료형: datetime, time, calender

date: 그레고리언 달력을 사용하여 년, 월, 일을 저장한다.

time: 하루 중 시간을 시간, 분, 초, 마이크로 초 단위로 저장한다.

datetime: 날짜와 시간을 같이 저장한다

timedelta: 두 datetime 값 간의 차이(일, 초, 마이크로초)를 표현한다

In [24]:
trip_df = pd.read_csv('./train.csv')
trip_df.drop(['id', 'vendor_id'], axis= 1,inplace = True)

간단한 시계열 포멧

%Y: 네자리 연도

%y: 두자리 연도

%m: 두자리 월 ex:01 은 1월

%d: 두자리 일 ex:03은 3일

%H: 24시간 형식 시간

%I: 12시간 형식 시간

%M: 두 자리 분

%S: 초

In [25]:
from datetime import datetime
# series에 있는 시계열 데이터는 현재 문자열 형태로 저장되어 있기 때문에 처리가 용이한 datetime 타입으로 바꾼다.
d2dt = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S') #datetime형으로 변환할 때 시계열 데이터의 형태를 맞춰서 넣어줘야한다.
pickup = trip_df.pickup_datetime.map(d2dt)
dropoff = trip_df.dropoff_datetime.map(d2dt)

In [28]:
#pickup 시간과 dropoff 시간의 간격을 초 단위로 바꾼다.
# datetime 형을 빼거나 더하면 timedelta 타입으로 바뀐다. timedelta 형은 시계열 형 데이터의 기간을 표현하는데 적합하다.
timegap = dropoff - pickup
tomsec = lambda x: x.seconds
timegap_sec = timegap.map(tomsec)

In [29]:
#timedelta형임을 확인할 수 있다.
type(timegap[0])

pandas._libs.tslibs.timedeltas.Timedelta

In [30]:
timegap_sec

0           455
1           663
2          2124
3           429
4           435
5           443
6           341
7          1551
8           255
9          1225
10         1274
11         1128
12         1114
13          260
14         1414
15          211
16         2316
17          731
18         1317
19          251
20          486
21          652
22          423
23         1163
24         2485
25         1283
26         1130
27          694
28          892
29         2331
           ... 
1458614     338
1458615     857
1458616     488
1458617     208
1458618     367
1458619     708
1458620    1162
1458621    2355
1458622     552
1458623     554
1458624     152
1458625    2002
1458626     244
1458627     880
1458628     351
1458629     601
1458630     549
1458631     342
1458632     777
1458633     979
1458634     972
1458635     237
1458636     800
1458637     760
1458638     414
1458639     778
1458640     655
1458641     764
1458642     373
1458643     198
Length: 1458644, dtype: 

In [31]:
#손님을 태운 시간과 내려준 시간의 차, 즉 trip_duration이 실제로는 이 데이터셋으로부터 바로 구할 수있다.
#위 데이터셋에서 trip_duration과 timegap_sec가 맞지 않는 4개의 데이터셋이 있는데 이는 시간이 잘못 기록되었음을 유추할 수 있다.
np.sum(timegap_sec != trip_df.trip_duration)

4